In [23]:
import json
from urllib.request import urlopen,quote
from urllib import parse
import pandas as pd 
import traceback
import os
import re
import numpy as np
import time
import datetime
import requests as rq
def get_location(address='上海市',impulse=5):
    query = {
     'key' : 'fGrKDFGpFcF8jKDTqDrSzdNazWq5zicB',
     'output':'json',
      }
    query['address']=address
    base = 'http://api.map.baidu.com/geocoder?'
    url = base+parse.urlencode(query)
    #print(url)
    for _ in range(3):
        try:
            doc=urlopen(url).read().decode('utf-8')
            jsonData=json.loads(doc)
            location=jsonData['result']['location']
            return(location)
        except Exception as e:
            print(e)
    return

from math import radians, cos, sin, asin, sqrt  
      
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）  
        """ 
        Calculate the great circle distance between two points  
        on the earth (specified in decimal degrees) 
        """  
        # 将十进制度数转化为弧度  
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
      
        # haversine公式  
        dlon = lon2 - lon1   
        dlat = lat2 - lat1   
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
        c = 2 * asin(sqrt(a))   
        r = 6371 # 地球平均半径，单位为公里  
        return (c * r )

In [20]:
search_location('地铁')

['121.318910', '31.193987', '虹桥火车站(地铁站)', '交通设施服务;地铁站;地铁站']

In [21]:
def gaode_location(keyword='',city='上海市',impulse=5):
    s=rq.session()
    #base_url='//restapi.amap.com/v3/place/text?key=2d0b7ddea6592e591fb3aa0ee0c8db30&keywords=%s&city=%s&children=1&offset=20&page=1&extensions=base'
    base_url='https://restapi.amap.com/v3/geocode/geo?address=%s&key=2d0b7ddea6592e591fb3aa0ee0c8db30'
    url=base_url%(city+keyword)
    #url=base_url%(keyword,city)
    for _ in range(3):
        try:
            web=s.get(url)
            address=web.json()['geocodes'][0]
            location=address['location'].split(',')+[address['formatted_address'],address['level']]
            return(location)
        except Exception as e:
            time.sleep(impulse)
            print(e)
    return

def search_location(keyword='',city='上海市',impulse=5):
    s=rq.session()
    base_url='https://restapi.amap.com/v3/place/text?keywords=%s&city=%s&offset=20&page=1&key=2d0b7ddea6592e591fb3aa0ee0c8db30&extensions=base'
    url=base_url%(keyword,city)
    for _ in range(3):
        try:
            web=s.get(url)
            address=web.json()['pois'][0]
            location=address['location'].split(',')+[address['name'],address['type']]
            return(location)
        except Exception as e:
            time.sleep(impulse)
            print(e)
    return
    
def correct_location(x):
    temp=search_location(x['fullname'])
    if temp :
        x['local']=temp
        x['lng']=temp[0]
        x['lat']=temp[1]
        x['level']=temp[3]
        x['formatname']=temp[2]
    return(x)

data=pd.read_csv('sold.csv')
# location=data[['name','district','region']].drop_duplicates()
# location['fullname']=location['district']+'区'+location['name']
# location['fullname']=location['fullname'].map(lambda x: str(x).replace('浦东区','浦东新区'))
# location['local']=location['fullname'].map(lambda x: gaode_location(x))
# location['lng']=location['local'].map(lambda x: x[0])
# location['lat']=location['local'].map(lambda x: x[1])
# location['level']=location['local'].map(lambda x: x[3])
# location['formatname']=location['local'].map(lambda x: x[2])

#location_test=location.loc[(location['level']=='区县') | (location['level']=='热点商圈'),]
#location_test=location_test.apply(correct_location,axis=1)
#location.loc[(location['level']=='区县') | (location['level']=='热点商圈'),['local','lng','lat','level','formatname']]=location_test[['local','lng','lat','level','formatname']]

In [4]:
new=pd.read_csv('new.csv')
data=pd.read_csv('sold.csv')
data=data.append(new,ignore_index=True)
data=data.sort_values('dealdate')
data=data.drop_duplicates(subset='link',keep='first')
data.to_csv('sold.csv',index=False)

In [31]:
#data=pd.read_csv('sold.csv')
data=data.sort_values('district')
data.index=range(0,len(data))
data=data.drop_duplicates(subset='link',keep='first')

data['space']=data['space'].map(lambda x: float(x.split('平')[0]) if str(x).find('平')>=0 else np.nan)
data['Price']=data['dealInfo'].map(lambda x: float(re.findall('\d+\.?\d*',x)[0]) if len(re.findall('\d+\.?\d*',x))==2 else np.nan)
data['Selldays']=data['dealInfo'].map(lambda x: float(re.findall('\d+\.?\d*',x)[1]) if len(re.findall('\d+\.?\d*',x))==2 else re.findall('\d+',x)[0])
data['Totalprice']=data['totalprice'].map(lambda x: float((x.split('万')[0])) if ('*' not in x and  "万"in x) else np.nan)
data['Unitprice']=data['unitprice'].map(lambda x: float(re.findall('\d+\.?\d*',x)[0]) if len(re.findall('\d+\.?\d*',x))==1 else np.nan)
data['date']=data['dealdate'].map(lambda x: x.replace('.','-').replace('近30天内成交',str(datetime.date.today()-datetime.timedelta(30))))
data['date']=data['date'].astype('datetime64[ns]')
data['zhuangxiu']=data['zhuangxiu'].map(lambda x: x.replace(' ','').replace('\xa0',''))

testdata=data.loc[(~data['Unitprice'].isnull()) & (~data['Price'].isnull()),]
discount=testdata['Totalprice'].mean()/testdata['Price'].mean()

data.loc[(data['Unitprice'].isnull()),'Totalprice']=data.loc[(data['Unitprice'].isnull()),'Price']*discount
data.loc[(data['Unitprice'].isnull()),'Unitprice']=data.loc[(data['Unitprice'].isnull()),
                                                            'Totalprice']/data.loc[(data['Unitprice'].isnull()),'space']*10000

hx=data.groupby('huxing').count().sort_values('name',ascending=False).index[0:9]
data['huxing']=data['huxing'].apply(lambda x: x if x in hx else '其他')
data['age']=data['position'].apply(lambda x: re.findall('\d*[0-9]',x)).apply(lambda x: datetime.datetime.today().year-int(x[1]) if len(x)==2 else np.nan)
data['year']=data['date'].astype(str).apply(lambda x: int(x.split('-')[0]))
data['floors']=data['position'].apply(lambda x: re.findall('\d*[0-9]层',x)).apply(lambda x: int(x[0].replace('层','')) if len(x)==1 else np.nan)
data['floors']=data['floors'].apply(lambda x : x if x <=40 else np.nan )
data['lifts']=data['floors'].apply(lambda x: '有电梯' if x>=7 else '无电梯')
data['buildyear']=data['position'].apply(lambda x: re.findall('\d*[0-9]年',x)).apply(lambda x: x[0] if len(x)==1 else np.nan)
year=data.groupby('buildyear').count().sort_values('name',ascending=False).index[0:30]
data['buildyear']=data['buildyear'].apply(lambda x : x if x in year else '其他')
data['metro']=data['HouseInfo'].apply(lambda x: re.findall('\d*[0-9]号线',str(x))).apply(lambda x: x[0] if len(x)==1 else np.nan)
metro=data.sort_values('metro')[['name','district','region','metro']].drop_duplicates()
metro=metro.drop_duplicates(subset=['name','region'])
data=pd.merge(data,metro,on=['name','district','region'],how='left',suffixes=['','s'])
data['车位']=data['position'].apply(lambda x :1 if '车位' in x else 0)

#data=data.set_index('date')



#location.to_csv('location.csv',index=False)


# location=pd.read_csv('location.csv')

# locat=data[['name','district','region']].drop_duplicates()
# locat['fullname']=locat['district']+'区'+locat['name']
# locat['fullname']=locat['fullname'].map(lambda x: str(x).replace('浦东区','浦东新区'))
# locat=locat.loc[~locat['fullname'].isin(location['fullname']),].copy()
# locat['local']=locat['fullname'].map(lambda x: get_location("上海市"+x))
# locat['lat']=locat['local'].map(lambda x: x['lat'] if x else np.nan)
# locat['lng']=locat['local'].map(lambda x: x['lng'] if x else np.nan)
# location=location.append(locat,ignore_index=True)

# location.to_csv('location.csv',index=True)

location=pd.read_excel('gaode_location.xlsx')
locat=data[['name','district','region']].drop_duplicates()
locat['fullname']=locat['district']+'区'+locat['name']
locat['fullname']=locat['fullname'].map(lambda x: str(x).replace('浦东区','浦东新区'))
locat=locat.loc[~locat['fullname'].isin(location['fullname']),].copy()
locat['local']=locat['fullname'].map(lambda x: gaode_location(x))
locat['lng']=locat['local'].map(lambda x: x[0])
locat['lat']=locat['local'].map(lambda x: x[1])
locat['level']=locat['local'].map(lambda x: x[3])
locat['formatname']=locat['local'].map(lambda x: x[2])
location=location.append(locat,ignore_index=True)
location.to_excel('gaode_location.xlsx')


data=pd.merge(data,location[['name','district','lat','lng']],on=['name','district'],how='left')
data=data.loc[~data['Unitprice'].isnull(),]
data=data.loc[data['车位']==0,]
data=data.loc[(data['Totalprice']>data['Totalprice'].quantile(0.001)) & (data['Totalprice']<data['Totalprice'].quantile(0.999)), ]                                                 
# data[['name','district','region','huxing','space','chaoxiang',
#        'zhuangxiu', 'lifts', 'position','metros', 'dealInfo', 'Totalprice',
#        'Unitprice', 'date', 'age','floors','buildyear','lat', 'lng']].to_csv('formatsold.csv',index=False)
formatdata=data[['name','district','region','huxing','space','chaoxiang',
       'zhuangxiu', 'lifts', 'position','metros', 'dealInfo', 'Totalprice',
       'Unitprice', 'date', 'age','floors','buildyear','lat', 'lng']].copy()
formatdata.columns=['小区','区域','街道','户型','面积','朝向','装修','电梯',
                    '楼层','地铁','交易信息','总价','单价','成交日期','楼龄','楼高','建筑年代','纬度','经度']
formatdata.to_csv('formatsold.csv',index=False)

In [7]:
formatdata.head()

,小区,区域,街道,户型,面积,朝向,装修,电梯,楼层,地铁,交易信息,总价,单价,成交日期,楼龄,楼高,建筑年代,纬度,经度
0,好世商业广场,嘉定,嘉定新城,3室2厅,106.89,南 北,精装,有电梯,低楼层(共20层) 2015年建板楼,NaN,挂牌423万成交周期119天,407.60405,38133.038614,2020-02-09,5.0,20.0,其他,31.3202,121.275
1,金鹤新城双佳翠庭,嘉定,江桥,1室1厅,58.00,南,毛坯,无电梯,中楼层(共5层) 2007年建板楼,13号线,挂牌232万成交周期93天,229.00000,39483.000000,2018-01-30,13.0,5.0,2007年,31.2475,121.313
2,中房公寓,嘉定,江桥,2室1厅,70.94,南,精装,有电梯,低楼层(共11层) 2005年建板楼,13号线,挂牌305万成交周期430天,300.00000,42290.000000,2018-01-30,15.0,11.0,2005年,31.2431,121.308
3,东方豪园(公寓),嘉定,嘉定新城,2室2厅,91.50,南,精装,有电梯,低楼层(共23层) 板楼,NaN,挂牌333万成交周期90天,325.00000,35520.000000,2018-01-30,NaN,23.0,其他,31.329,121.277
4,利金国际,嘉定,丰庄,1室0厅,45.71,北,其他,有电梯,中楼层(共26层) 塔楼,NaN,挂牌70万成交周期375天,68.00000,14877.000000,2016-07-26,NaN,26.0,其他,31.2516,121.367


In [32]:
data['year']=data['date'].apply(lambda x: str(x).split('-')[0])
data=data.loc[(data['Totalprice']>data['Totalprice'].quantile(0.005)) & (data['Totalprice']<data['Totalprice'].quantile(0.995)), ]
testdata=data[['name','district','region','space',
        'lifts', 'Totalprice', 'Unitprice', 'year', 'age','floors','metros']].copy()
testdata['Unitprice']=testdata['Unitprice']/10000
testdata['metro']=testdata['metros'].apply(lambda x: 1 if isinstance(x,str) else 0)
testdata['lift']=testdata['lifts'].apply(lambda x: 0 if x=="无电梯" else 1)
xiaoqu=testdata.groupby(['name','district']).count()[['region']].join(pd.pivot_table(testdata,index='name',values=['space','age','metro','lift'],
                     aggfunc=np.mean))
yearly=pd.pivot_table(testdata.loc[~testdata['year'].isin(['2015','2019','2020']),],index='name',columns='year',
                     values=['Totalprice','Unitprice'],aggfunc=np.mean)
yearly.columns=['Totalprice2016','Totalprice2017','Totalprice2018','Unitprice2016','Unitprice2017','Unitprice2018']
xiaoqu=xiaoqu.join(yearly)
xiaoqu=xiaoqu.rename(columns={'region':'Count'})
xiaoqu=xiaoqu.query('Count>2')
xiaoqu=xiaoqu.loc[~((xiaoqu['Totalprice2016'].isnull()) & (xiaoqu['Totalprice2017'].isnull())),]
xiaoqu=xiaoqu.loc[~((xiaoqu['Totalprice2016'].isnull()) & (xiaoqu['Totalprice2018'].isnull())),]
xiaoqu=xiaoqu.loc[~((xiaoqu['Totalprice2017'].isnull()) & (xiaoqu['Totalprice2018'].isnull())),]
xiaoqu=xiaoqu.reset_index()
xiaoqu.columns=['小区名称','区域','已售','楼龄','电梯','地铁','面积','总价1','总价2','总价3','单价1','单价2','单价3']
xiaoqu.to_csv('xiaoqu.csv',index=False)

In [10]:
data['year']=data['date'].apply(lambda x: str(x).split('-')[0])
data=data.loc[(data['Totalprice']>data['Totalprice'].quantile(0.005)) & (data['Totalprice']<data['Totalprice'].quantile(0.995)), ]
testdata=data[['name','district','region','space',
        'lifts', 'Totalprice', 'Unitprice', 'year', 'age','floors','metros']].copy()
testdata['Unitprice']=testdata['Unitprice']/10000
testdata['metro']=testdata['metros'].apply(lambda x: 1 if isinstance(x,str) else 0)
testdata['lift']=testdata['lifts'].apply(lambda x: 0 if x=="无电梯" else 1)
xiaoqu=testdata.groupby(['name','district']).count()[['region']].join(pd.pivot_table(testdata,index='name',values=['space','age','metro','lift'],
                     aggfunc=np.mean))

In [12]:
yearly=pd.pivot_table(testdata,index='name',columns='year',
                     values=['Totalprice','Unitprice'],aggfunc=np.mean)

In [15]:
data=pd.read_excel('gaode_location.xlsx')

In [17]:
pd.read_excel('subway.xlsx')

,Line,Lng,Lat,StationName,City
0,3号线,121.479574,31.395315,宝杨路,上海
1,3号线,121.488247,31.381302,水产路,上海
2,3号线,121.492818,31.370928,淞滨路,上海
3,3号线,121.500390,31.345135,淞发路,上海
4,3号线,121.485082,31.305546,江湾镇,上海
5,3号线,121.482429,31.281247,赤峰路,上海
6,3号线,121.480202,31.259886,东宝兴路,上海
7,3号线,121.458219,31.249538,上海火车站,上海
8,3号线,121.441001,31.254568,中潭路,上海
9,3号线,121.413257,31.232307,金沙江路,上海
